# Baseline EEG Analysis

This notebook is geared towards looking at resting and concentration data and identifying the following:

* How do we extract the mean and median data of each EEG file?
* How can we conduct some form of data analysis to identify if there's a significant difference between the data across different trial types? In particular:
    * Is there a significant difference between the resting state of the HMD vs non-HMD?
    * Is there a significant difference between the 2D and 3D red ball experiment?

There is a question here - do we use the entire 30 seconds and their mean/median, or do we split the segments into windows and conduct the analysis via overlapping windows per participant? Whichever the case, we need the rest and concentration data across ALL participants, not just singular ones.

Trial data for all participants can be found locally in `./samples/rest_conc_data/` directory. We take the data from each participant and flattened them, labeling them `P1` to `P16`. We'll conduct the study across all participants.

The outputs of this analysis are thus:

1. A singular CSV file containing the mean and median of each participant. This one will be derived from the total mean and median across the entire 30 seconds.

# Setup

In [16]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# Required for converting between local timestamps and unix timestamp
import datetime

# Required for aligning timestamps
import shutil

# Required for detecting files inside of a provided directory
from os import listdir
from os.path import isfile, join

# For Wilcoxon Signed Rank Test
from scipy.stats import wilcoxon

## Helper Functions

In [2]:
""" === HELPER FUNCTION === """
# Converts the format of a local timestamp into unix seconds. Requires the datetime package
def timestamp_to_unix_seconds(x):
    date_format = datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
    unix_seconds = datetime.datetime.timestamp(date_format)
    return unix_seconds

# Converts the format of a local timestamp into unix milliseconds. 
# Requires the datatime package, and relies on `timestamp_to_unix_seconds()` function.
def timestamp_to_unix_milliseconds(x):
    unix_seconds = timestamp_to_unix_seconds(x)
    unix_milliseconds = int(unix_seconds * 1000)
    return unix_milliseconds

# Instructions

## 1. Reading Raw EEG Data

This script was copied from `align_video_eeg.ipynb` for simplicity. The one change we make is that we don't set any start or end unix millisecond - instead, we auto-remove any rows that are within 5 seconds of the start and end of the data itself.

In [3]:
# Reads the raw eeg file from the provided file URL, converts the timestamps, and filters rows.
# Note that `start_milliseconds` is derived from unix milliseconds, not relative milliseconds. AKA provide a unix millisecond that represents when the trial starts.
def Get_Raw_EEG(
        src:str, 
        start_buffer_milliseconds:int = 5000,
        end_buffer_milliseconds:int = 5000,
        save_filename:str=None, 
        print_debug:bool=False
):
    
    # Read the EEG file
    if print_debug: print("Reading SRC csv file...")
    df = pd.read_csv(src)
    n_rows = len(df.index)
    
    # Filter by removing any rows that have the timestamp as null and the battery amount indicator, which is not useful here.
    if print_debug: print("Removing rows with null timestamps and battery...")
    df = df[~df['TimeStamp'].isna()]
    df = df[~df['Battery'].isna()]
    n_removed_rows = n_rows - len(df.index)
    
    # Convert the "TimeStamp" column into unix milliseconds
    if print_debug: print("Converting timestamps to unix milliseconds...")
    df['unix_ms'] = df['TimeStamp'].apply(lambda x: int(timestamp_to_unix_milliseconds(x)))
    
    # From the start of recording this file, get the "relative" unix milliseconds.
    if print_debug: print("Deriving relative unix milliseconds...")
    raw_start_unix_milliseconds = df['unix_ms'].iloc[0]
    df['rel_unix_ms'] = df['unix_ms'] - raw_start_unix_milliseconds
    
    # Filter further: remove any rows that are within 5 seconds of either the start or end.
    # This filtering is based on the first and last values of the `rel_unix_ms` column, for simplicity.
    if print_debug: print("Determining end unix milliseconds based on either duration or last row, filtering rows...")
    start_unix_milliseconds = start_buffer_milliseconds
    end_unix_milliseconds = df['rel_unix_ms'].iloc[-1] - end_buffer_milliseconds
    df = df[df['rel_unix_ms'].between(start_unix_milliseconds, end_unix_milliseconds)]
    
    # Given the new filtered rows, get the trial milliseconds instead
    if print_debug: print("Deriving trial milliseconds...")
    df['trial_ms'] = df['rel_unix_ms'] - df['rel_unix_ms'].iloc[0]
    
    # Final steps
    if print_debug: print(f"\nFinished processing {src}!")
    final_start_milliseconds = df['trial_ms'].iloc[0]
    final_end_milliseconds = df['trial_ms'].iloc[-1]
    final_duration_milliseconds = final_end_milliseconds - final_start_milliseconds
    
    if save_filename is not None:
        # Deriving file path from provided src url
        src_directory = os.path.dirname(src)
        save_name = os.path.join(src_directory, save_filename)
        df.to_csv(save_name)
        if print_debug: print(f"Saving resulting eeg to {save_name}")
    
    if print_debug:
        print(f"\t- # raw rows: {n_rows}")
        print(f"\t- # filtered rows based on NA timestamps: {n_removed_rows}")
        print(f"\t- Final number of rows: {len(df.index)}")
        print(f"\t- Time removed since start of recording: {final_start_milliseconds - raw_start_unix_milliseconds}")
        print(f"\t- Trial start and end: {final_start_milliseconds} - {final_end_milliseconds}")
        print(f"\t- Trial duration: {final_duration_milliseconds}")
              
    return df, (start_unix_milliseconds, end_unix_milliseconds, final_duration_milliseconds)

In [6]:
# Sample Case
df, details = Get_Raw_EEG('./samples/rest_conc_data/p1-conc-2d.csv', print_debug=True)
df

Reading SRC csv file...
Removing rows with null timestamps and battery...
Converting timestamps to unix milliseconds...
Deriving relative unix milliseconds...
Determining end unix milliseconds based on either duration or last row, filtering rows...
Deriving trial milliseconds...

Finished processing ./samples/rest_conc_data/p1-conc-2d.csv!
	- # raw rows: 48
	- # filtered rows based on NA timestamps: 4
	- Final number of rows: 34
	- Time removed since start of recording: -1728505980082
	- Trial start and end: 0 - 33010
	- Trial duration: 33010


,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HeadBandOn,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10,Battery,Elements,unix_ms,rel_unix_ms,trial_ms
8,2024-10-09 16:33:05.082,0.474925,0.160934,-0.159773,0.447115,0.866769,0.103431,-0.340348,0.444156,1.297534,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505985082,5000,0
9,2024-10-09 16:33:06.159,0.380913,0.056486,-0.287304,0.737206,0.599921,-0.151852,-0.214520,0.731053,1.552879,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505986159,6077,1077
10,2024-10-09 16:33:07.126,0.207187,0.289955,-0.146099,0.505423,0.334001,-0.004268,-0.123826,0.524455,1.525738,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505987126,7044,2044
11,2024-10-09 16:33:08.356,0.320366,0.407583,0.078954,0.280365,0.507494,-0.002677,-0.237650,0.404424,1.339155,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505988356,8274,3274
12,2024-10-09 16:33:09.145,0.737952,0.786047,0.336173,1.207276,0.817108,0.333303,-0.323100,0.611070,1.139489,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505989145,9063,4063
13,2024-10-09 16:33:10.168,0.924413,0.809755,0.570512,1.580261,0.894782,0.328667,-0.160741,0.665484,0.897806,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505990168,10086,5086
14,2024-10-09 16:33:11.177,0.113235,0.480951,-0.479515,0.703378,0.543990,0.312574,-0.333316,0.579892,1.374370,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505991177,11095,6095
15,2024-10-09 16:33:12.126,0.443669,0.808578,-0.323182,0.779253,0.785850,0.527217,-0.299351,0.657347,1.429244,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505992126,12044,7044
16,2024-10-09 16:33:13.121,0.636691,0.603974,-0.121113,0.786224,0.597553,0.249094,-0.249728,0.552090,1.193694,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505993121,13039,8039
17,2024-10-09 16:33:14.083,0.790157,0.016351,0.175998,0.521910,0.760035,0.624187,0.101178,0.655986,1.192112,...,1.0,1.0,1.0,1.0,1.0,75.0,NaN,1728505994083,14001,9001


To this end, we just now have to be able to read an EEG file, classify it based on filename (i.e. `p1` = "Participant 1, `conc` vs `rest`, `novr` vs `vr` and `2d` vs `3d`). Not too hard... probably.

In [7]:
src_dir = './samples/rest_conc_data/'

# This contains the raw EEG filenames
files = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]

# Based on raw filename, which we have to individually parse - 
#   we categorize each file by participant, trial type, and subtrial type.

all_dfs = []
for f in files:
    
    # Extract the full relative URL to the file itself
    fullF = join(src_dir, f)
    print(f"Reading {fullF}...")
    
    # Split the EEG filename into segments. Exit early if we can't parse their name, for some reason
    segments = os.path.splitext(os.path.basename(f))[0].split('-')
    if len(segments)!=3:
        print(f"\tERROR: Cannot parse {f} - properly incorrectly named")
        continue
    
    # Let's read the EEG data itself. We should be able to extract a DF from it
    df, details = Get_Raw_EEG(fullF, print_debug=False)
    
    # Since we already have a DF in of itself, let's just append the data itself to this existing df
    df['participant'] = segments[0]
    df['task'] = segments[1]
    df['task_type'] = segments[2]
    
    # Save the resulting df into our `all_dfs` list
    all_dfs.append(df)

# With all dfs saved, let's concat them all
rest_conc_df = pd.concat(all_dfs)
rest_conc_df

Reading ./samples/rest_conc_data/p1-conc-2d.csv...
Reading ./samples/rest_conc_data/p1-conc-3d.csv...
Reading ./samples/rest_conc_data/p1-rest-novr.csv...
Reading ./samples/rest_conc_data/p1-rest-vr.csv...
Reading ./samples/rest_conc_data/p10-conc-2d.csv...
Reading ./samples/rest_conc_data/p10-conc-3d.csv...
Reading ./samples/rest_conc_data/p10-rest-novr.csv...
Reading ./samples/rest_conc_data/p10-rest-vr.csv...
Reading ./samples/rest_conc_data/p11-conc-2d.csv...
Reading ./samples/rest_conc_data/p11-conc-3d.csv...
Reading ./samples/rest_conc_data/p11-rest-novr.csv...
Reading ./samples/rest_conc_data/p11-rest-vr.csv...
Reading ./samples/rest_conc_data/p12-conc-2d.csv...
Reading ./samples/rest_conc_data/p12-conc-3d.csv...
Reading ./samples/rest_conc_data/p12-rest-novr.csv...
Reading ./samples/rest_conc_data/p12-rest-vr.csv...
Reading ./samples/rest_conc_data/p13-conc-2d.csv...
Reading ./samples/rest_conc_data/p13-conc-3d.csv...
Reading ./samples/rest_conc_data/p13-rest-novr.csv...
Readin

,TimeStamp,Delta_TP9,Delta_AF7,Delta_AF8,Delta_TP10,Theta_TP9,Theta_AF7,Theta_AF8,Theta_TP10,Alpha_TP9,...,HSI_TP10,Battery,Elements,unix_ms,rel_unix_ms,trial_ms,participant,task,task_type,AUX_LEFT
8,2024-10-09 16:33:05.082,0.474925,0.160934,-0.159773,0.447115,0.866769,0.103431,-0.340348,0.444156,1.297534,...,1.0,75.0,NaN,1728505985082,5000,0,p1,conc,2d,NaN
9,2024-10-09 16:33:06.159,0.380913,0.056486,-0.287304,0.737206,0.599921,-0.151852,-0.214520,0.731053,1.552879,...,1.0,75.0,NaN,1728505986159,6077,1077,p1,conc,2d,NaN
10,2024-10-09 16:33:07.126,0.207187,0.289955,-0.146099,0.505423,0.334001,-0.004268,-0.123826,0.524455,1.525738,...,1.0,75.0,NaN,1728505987126,7044,2044,p1,conc,2d,NaN
11,2024-10-09 16:33:08.356,0.320366,0.407583,0.078954,0.280365,0.507494,-0.002677,-0.237650,0.404424,1.339155,...,1.0,75.0,NaN,1728505988356,8274,3274,p1,conc,2d,NaN
12,2024-10-09 16:33:09.145,0.737952,0.786047,0.336173,1.207276,0.817108,0.333303,-0.323100,0.611070,1.139489,...,1.0,75.0,NaN,1728505989145,9063,4063,p1,conc,2d,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-11-20 11:23:31.372,1.503729,0.779777,0.795692,1.184934,0.354691,0.209845,0.483923,0.407861,1.067565,...,1.0,100.0,NaN,1732119811372,29000,23991,p9,rest,vr,NaN
31,2024-11-20 11:23:32.373,1.503729,0.670556,0.545350,0.940302,0.354691,0.204985,0.422719,0.245162,1.067565,...,1.0,100.0,NaN,1732119812373,30001,24992,p9,rest,vr,NaN
32,2024-11-20 11:23:33.378,1.503729,0.513105,0.707788,0.507392,0.354691,0.087379,0.387460,0.344344,1.067565,...,1.0,100.0,NaN,1732119813378,31006,25997,p9,rest,vr,NaN
33,2024-11-20 11:23:34.371,0.896890,0.539139,0.770070,0.726940,0.537933,0.162738,0.313993,0.508986,0.589024,...,1.0,100.0,NaN,1732119814371,31999,26990,p9,rest,vr,NaN


## Extracting Mean and Median

What's important to note is that because we're already dealing with PSD data, we already did some form of windowing. In early participants, we had a hamming window of 1sec. In more recent trials, that discretization was around 0.5sec. So I don't think it's necessarily prudent to use windows of time here either...

For the moment, let's focus on just getting the mean and median of each data. This is a relatively simple process - we have an `rest_conc_df` that contains all our data. We just need to group them when we do the mean and median calculation, which comes with pandas.

In [9]:
rest_conc_median_df = rest_conc_df.groupby(["participant", "task", "task_type"], as_index=False).agg({
    'Delta_AF7':'median', 
    'Delta_AF8':'median',
    'Delta_TP9':'median',
    'Delta_TP10':'median',
    'Theta_AF7':'median', 
    'Theta_AF8':'median',
    'Theta_TP9':'median',
    'Theta_TP10':'median',
    'Alpha_AF7':'median', 
    'Alpha_AF8':'median',
    'Alpha_TP9':'median',
    'Alpha_TP10':'median',
    'Beta_AF7':'median', 
    'Beta_AF8':'median',
    'Beta_TP9':'median',
    'Beta_TP10':'median',
    'Gamma_AF7':'median', 
    'Gamma_AF8':'median',
    'Gamma_TP9':'median',
    'Gamma_TP10':'median'
})
rest_conc_median_df

,participant,task,task_type,Delta_AF7,Delta_AF8,Delta_TP9,Delta_TP10,Theta_AF7,Theta_AF8,Theta_TP9,...,Alpha_TP9,Alpha_TP10,Beta_AF7,Beta_AF8,Beta_TP9,Beta_TP10,Gamma_AF7,Gamma_AF8,Gamma_TP9,Gamma_TP10
0,p1,conc,2d,0.383444,-0.119841,0.434764,0.489830,0.170884,-0.132327,0.706354,...,1.349438,1.298825,0.461137,0.682781,0.655964,0.602923,0.194695,0.390604,-0.009749,0.111069
1,p1,conc,3d,0.135368,0.020055,0.442325,0.405492,0.115379,-0.161341,0.414515,...,1.122643,0.970189,0.224458,0.564174,0.451997,0.487855,-0.190019,0.322103,-0.027758,0.066373
2,p1,rest,novr,0.516913,0.386579,0.441442,0.578261,0.030591,0.085284,0.657871,...,1.316836,1.132963,0.040447,0.032539,0.622401,0.662241,-0.382675,-0.392358,-0.058598,0.100376
3,p1,rest,vr,0.774858,0.910769,1.247597,1.163619,0.410288,0.478153,0.887971,...,1.575208,1.528088,0.267301,0.603266,0.677705,0.716333,-0.146457,0.344795,0.126304,0.232457
4,p10,conc,2d,-0.164429,-0.282928,0.117532,0.122240,-0.220437,-0.279917,0.053198,...,0.323998,0.431278,-0.129662,-0.119846,0.206682,0.417130,-0.398908,-0.452710,-0.041272,0.130974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,p8,rest,vr,0.543182,0.110921,0.543611,0.522944,0.214688,-0.064793,0.474985,...,1.265483,1.182902,0.558801,0.515710,0.647085,0.702091,0.297922,0.389066,0.379651,0.401919
60,p9,conc,2d,0.347178,0.252543,0.568807,0.377026,0.076584,0.120117,0.388581,...,0.812521,0.528480,0.398279,0.148637,0.470390,0.295794,0.191484,-0.225210,0.114545,-0.068936
61,p9,conc,3d,0.203532,0.159101,0.795078,0.325956,0.086331,0.016348,0.459896,...,0.731070,0.630309,0.281891,0.044250,0.496778,0.419076,-0.006778,-0.350216,0.091095,0.206523
62,p9,rest,novr,0.642916,0.614031,1.003289,0.910563,0.293159,0.324359,0.579543,...,0.978162,0.954907,0.165522,0.102898,0.381729,0.335010,-0.207204,-0.249300,-0.096757,-0.121563


# Measuring Statistical Significance for Condition Differences

The idea of this entire analysis is to identify if the concentration condition produces a difference. AKA if the introduction of a red-ball concentration test induces people to create differences in concentration level. Of course, this assumes that we have some paired data (i.e. for each participant, we have a "before" and "after" condition). This, of course, is true - we have the rest state come before the concentration state in all participants. However, this kind of test assumes that our hypothesis is that "the red ball test resulted in a statistically significant difference in concentration levels among participants". The intention is subtle - in our original question, we posit whether there even is a difference, while the statistical significant test posits whether the introduction of a new condition __creates__ a difference. Subtle, to be sure, but something we must acknowledge. This means there's a dependence between the two conditions - a before and after, in most cases.

Source: https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/how-to-conduct-the-wilcox-sign-test/

There might be some ways to go about this. One can try to identify which data corresponds with their order, then flip them accordingly. Another might be to just find the absolute differences so that we only care about the magnitude of the differences. This might be the direction we want to go down in the end - after all, our null hypothesis is that the introduction of the red ball concentration task does not induce a difference in eeg log power level. Since the log power effect is applied to all data columns and rows, seeing changes in that will also echo changes in the true absolute power of the eeg data.

For now, let's go with Wilcoxon Signed Rank Test, which allows us to measure this difference without any assumption of normality in data points. We just have to account for the absolute difference in the condition instead.

Python Code: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html

Note that our alternative hypothesis, the one we want to support, is that 

## Statistic Difference for the Influence of VR HMD Presence during Resting

In [10]:
# First test: measure difference between VR and NoVR rest state
rest_median_df = rest_conc_median_df[rest_conc_median_df['task'] == 'rest']
rest_median_df

,participant,task,task_type,Delta_AF7,Delta_AF8,Delta_TP9,Delta_TP10,Theta_AF7,Theta_AF8,Theta_TP9,...,Alpha_TP9,Alpha_TP10,Beta_AF7,Beta_AF8,Beta_TP9,Beta_TP10,Gamma_AF7,Gamma_AF8,Gamma_TP9,Gamma_TP10
2,p1,rest,novr,0.516913,0.386579,0.441442,0.578261,0.030591,0.085284,0.657871,...,1.316836,1.132963,0.040447,0.032539,0.622401,0.662241,-0.382675,-0.392358,-0.058598,0.100376
3,p1,rest,vr,0.774858,0.910769,1.247597,1.163619,0.410288,0.478153,0.887971,...,1.575208,1.528088,0.267301,0.603266,0.677705,0.716333,-0.146457,0.344795,0.126304,0.232457
6,p10,rest,novr,0.006171,-0.034561,0.536725,0.526666,-0.175682,-0.219768,0.216320,...,0.923631,0.799700,-0.052626,-0.089758,0.533508,0.435110,-0.134497,-0.396829,0.096569,0.042066
7,p10,rest,vr,0.089051,0.194414,0.425545,0.478274,-0.231298,-0.239843,0.321910,...,1.091467,1.058145,-0.144140,-0.086383,0.516157,0.602099,-0.390401,-0.463022,0.311559,0.368716
10,p11,rest,novr,-0.029737,0.112077,0.616287,0.512398,-0.098663,0.007546,0.563050,...,1.296757,1.129884,-0.117697,-0.081757,0.422325,0.379908,-0.355146,-0.362279,-0.078638,-0.028740
11,p11,rest,vr,0.258711,0.186629,0.551467,0.493812,-0.013016,-0.112698,0.542152,...,1.175434,1.044895,-0.058403,-0.146354,0.465480,0.402896,-0.436810,-0.435121,0.102743,0.049536
14,p12,rest,novr,0.031049,0.070738,0.296560,0.422225,-0.111751,-0.151310,0.322974,...,0.880347,1.002078,-0.075430,-0.098510,0.377353,0.447171,-0.399531,-0.456301,-0.209085,-0.141446
15,p12,rest,vr,0.134644,0.130171,0.622836,0.526869,-0.080064,-0.000259,0.432019,...,0.937336,0.965718,-0.053451,-0.032680,0.438938,0.493390,-0.427828,-0.351993,-0.034626,0.044792
18,p13,rest,novr,0.325527,0.719035,0.476462,0.417983,-0.276978,-0.219503,0.043521,...,0.558439,0.604324,-0.168036,-0.208572,0.246173,0.266323,-0.519407,-0.580277,-0.111109,-0.051101
19,p13,rest,vr,0.267668,0.571368,0.373789,0.363341,-0.229343,-0.351244,0.014483,...,0.625306,0.691076,-0.168451,-0.222695,0.197408,0.391278,-0.482826,-0.593158,-0.094253,0.226216


In [15]:
numeric_cols = [
    'Delta_AF7', 
    'Delta_AF8',
    'Delta_TP9',
    'Delta_TP10',
    'Theta_AF7', 
    'Theta_AF8',
    'Theta_TP9',
    'Theta_TP10',
    'Alpha_AF7', 
    'Alpha_AF8',
    'Alpha_TP9',
    'Alpha_TP10',
    'Beta_AF7', 
    'Beta_AF8',
    'Beta_TP9',
    'Beta_TP10',
    'Gamma_AF7', 
    'Gamma_AF8',
    'Gamma_TP9',
    'Gamma_TP10'
]

def subtract_rows(group):
    return group[numeric_cols].iloc[1] - group[numeric_cols].iloc[0]

rest_median_diff_df = rest_median_df.groupby(['participant'], as_index=False).apply(subtract_rows)
rest_median_diff_df

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\2183217506.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rest_median_diff_df = rest_median_df.groupby(['participant'], as_index=False).apply(subtract_rows)


,participant,Delta_AF7,Delta_AF8,Delta_TP9,Delta_TP10,Theta_AF7,Theta_AF8,Theta_TP9,Theta_TP10,Alpha_AF7,...,Alpha_TP9,Alpha_TP10,Beta_AF7,Beta_AF8,Beta_TP9,Beta_TP10,Gamma_AF7,Gamma_AF8,Gamma_TP9,Gamma_TP10
0,p1,0.257945,0.524190,0.806155,0.585359,0.379697,0.392869,0.230101,0.374950,0.285707,...,0.258372,0.395126,0.226855,0.570728,0.055304,0.054092,0.236218,0.737153,0.184901,0.132081
1,p10,0.082880,0.228975,-0.111180,-0.048392,-0.055617,-0.020076,0.105591,-0.092450,0.074081,...,0.167837,0.258446,-0.091513,0.003375,-0.017351,0.166989,-0.255905,-0.066193,0.214990,0.326650
2,p11,0.288449,0.074551,-0.064820,-0.018586,0.085648,-0.120243,-0.020898,-0.017237,0.103663,...,-0.121323,-0.084989,0.059294,-0.064597,0.043154,0.022988,-0.081664,-0.072842,0.181381,0.078276
3,p12,0.103595,0.059433,0.326276,0.104644,0.031686,0.151051,0.109045,-0.124831,0.028271,...,0.056990,-0.036359,0.021978,0.065830,0.061585,0.046219,-0.028296,0.104308,0.174459,0.186238
4,p13,-0.057859,-0.147667,-0.102673,-0.054642,0.047635,-0.131741,-0.029038,-0.082040,0.099974,...,0.066867,0.086752,-0.000416,-0.014123,-0.048765,0.124955,0.036581,-0.012880,0.016856,0.277316
5,p14,0.027053,-0.005274,-0.178360,-0.469978,-0.097817,-0.108919,-0.211739,-0.454288,-0.009630,...,-0.146034,-0.222867,-0.004723,-0.051584,-0.034216,-0.139667,0.008103,-0.105309,-0.031335,-0.080147
6,p15,-0.011344,-0.281902,0.097842,-0.243631,-0.012424,-0.051635,0.056014,-0.358682,0.128996,...,-0.005642,-0.076202,0.338968,0.275802,-0.058619,-0.147747,0.445507,0.438695,-0.287668,-0.041847
7,p16,0.023772,0.083774,-0.108351,-0.041893,0.140959,0.149901,-0.119109,-0.039142,0.174437,...,-0.113225,-0.139490,0.106465,0.140741,-0.002910,-0.023236,0.223279,0.140379,0.023452,0.022734
8,p2,0.207114,-0.253950,0.101442,0.201989,0.166364,-0.233555,0.017009,0.147255,0.088083,...,-0.051274,0.025036,0.008532,0.120840,0.030638,0.146902,0.079028,0.333557,0.116261,0.120217
9,p3,0.317590,0.276392,-0.038325,-0.868263,0.162149,0.176392,0.012990,-0.522301,-0.005159,...,-0.212033,-0.042258,0.251991,0.322461,0.038607,-0.202824,0.423147,0.459727,0.253841,0.052711


In [36]:
def ConductWilcoxon(cols, df):
    for colname in cols:
        d = df[colname].tolist()
        twoside_res = wilcoxon(d)
        greater_res = wilcoxon(d, alternative='greater')
        less_res = wilcoxon(d, alternative='less')
        print(f"{colname}\t2Sided: {twoside_res.statistic}/{twoside_res.pvalue} \t|Greater: {greater_res.statistic}/{greater_res.pvalue}  \t|Less: {less_res.statistic}/{less_res.pvalue}")
        
ConductWilcoxon(numeric_cols, rest_median_diff_df)

Delta_AF7	2Sided: 26.0/0.02899169921875 	|Greater: 110.0/0.014495849609375  	|Less: 110.0/0.987518310546875
Delta_AF8	2Sided: 38.0/0.129730224609375 	|Greater: 98.0/0.0648651123046875  	|Less: 98.0/0.9416656494140625
Delta_TP9	2Sided: 60.0/0.705718994140625 	|Greater: 76.0/0.3528594970703125  	|Less: 76.0/0.6657257080078125
Delta_TP10	2Sided: 55.0/0.528167724609375 	|Greater: 55.0/0.752288818359375  	|Less: 55.0/0.2640838623046875
Theta_AF7	2Sided: 38.0/0.129730224609375 	|Greater: 98.0/0.0648651123046875  	|Less: 98.0/0.9416656494140625
Theta_AF8	2Sided: 51.0/0.40374755859375 	|Greater: 85.0/0.201873779296875  	|Less: 85.0/0.8122711181640625
Theta_TP9	2Sided: 46.0/0.274444580078125 	|Greater: 90.0/0.1372222900390625  	|Less: 90.0/0.8738861083984375
Theta_TP10	2Sided: 46.0/0.274444580078125 	|Greater: 46.0/0.8738861083984375  	|Less: 46.0/0.1372222900390625
Alpha_AF7	2Sided: 15.0/0.004180908203125 	|Greater: 121.0/0.0020904541015625  	|Less: 121.0/0.998321533203125
Alpha_AF8	2Sided: 44

Analaysis of the Wilcoxon Signed Rank Test shows that, under the asusmption of a 5% confidence level, we must reject the null hypothesis that the wearing of the HMD does not affect the log power of the EEG signals for the selected groups:

* Delta AF7
* Alpha AF7
* Beta AF8
* Gamma TP9
* Gamma TP10

If we explore the idea that the median of the differences is greater than 0, then we can confirm this hypothesis (thus reject the null hypothesis that the median difference is not greater than 0) for the following channels and bands:

* Delta AF7
* Alpha AF7
* Beta AF7
* Beta AF8
* Gamma TP9
* Gamma TP10

And naturally, we cannot reject the null hypothesis that the median of the differences is not smaller than 0, given that all pvalues are above the 5% confidence threshold.

__What does this mean?__ I would say that, given that we are primarily interested in the Alpha, Beta, and Gamma bands, the fact that we DO get a response from those bands is somewhat promising. What is a little weird though is that the alpha and delta only have AF7, beta only has AF8, and gamma is... gamma for TP9 and TP10. We don't really want to explore TP9 or TP10, so this is a little curious. I think though it can be said at least that the EEG data for TP9 and TP10 is at least stable. Or at least I would like to think so. They're certainly more stable than the real-world condition.

## Statistic Difference for the Influence of 2D and 3D Red Ball During Concentration

In [37]:
conc_median_df = rest_conc_median_df[rest_conc_median_df['task'] == 'conc']
conc_median_diff_df = conc_median_df.groupby(['participant'], as_index=False).apply(subtract_rows)
ConductWilcoxon(numeric_cols, conc_median_diff_df)

Delta_AF7	2Sided: 65.0/0.899932861328125 	|Greater: 71.0/0.4499664306640625  	|Less: 71.0/0.5698699951171875
Delta_AF8	2Sided: 58.0/0.632171630859375 	|Greater: 78.0/0.3160858154296875  	|Less: 78.0/0.7017059326171875
Delta_TP9	2Sided: 59.0/0.668548583984375 	|Greater: 59.0/0.6839141845703125  	|Less: 59.0/0.3342742919921875
Delta_TP10	2Sided: 45.0/0.252227783203125 	|Greater: 45.0/0.8843994140625  	|Less: 45.0/0.1261138916015625
Theta_AF7	2Sided: 61.0/0.743560791015625 	|Greater: 75.0/0.3717803955078125  	|Less: 75.0/0.6471405029296875
Theta_AF8	2Sided: 59.0/0.668548583984375 	|Greater: 59.0/0.6839141845703125  	|Less: 59.0/0.3342742919921875
Theta_TP9	2Sided: 63.0/0.820892333984375 	|Greater: 73.0/0.4104461669921875  	|Less: 73.0/0.6090240478515625
Theta_TP10	2Sided: 28.0/0.03863525390625 	|Greater: 28.0/0.9832305908203125  	|Less: 28.0/0.019317626953125
Alpha_AF7	2Sided: 62.0/0.781951904296875 	|Greater: 74.0/0.3909759521484375  	|Less: 74.0/0.6282196044921875
Alpha_AF8	2Sided: 35.0

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\2494310905.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  conc_median_diff_df = conc_median_df.groupby(['participant'], as_index=False).apply(subtract_rows)


Huzzah! A damn easy thing. We cannot reject the null hypothesis that the 3D condition is different from the 2D condition. All for.... two cases: __Theta TP10__ and __Alpha TP10__.

If the null hypothesis is that the median difference is greater than 0, then rest assured that we cannot reject that null hypothesis either.

If the null hypothesis is that the median difference is smaller than 0, then we only can reject that null hypothesis for these cases: __Theta TP10__, __Alpha AF8__, __Alpha TP10__.

What does this mean? Note that Theta is the only frequency band that has some response here, so far. Furthermore, it's specifically in the TP10 channel. The same response is seen in Alpha TP10. What's very interesting is Alpha AF8, which also does not show up when we looked at the rest state conditions. Interesting...

## Comparing No-HMD Rest and 3D

In [51]:
novr_3d_median_df = rest_conc_median_df[rest_conc_median_df['task_type'].isin(["novr","3d"])]
# Re-sort so that the rest comes before the concentration task
novr_3d_median_sorted_df = novr_3d_median_df.sort_values(['participant','task'],ascending=False).groupby('participant').head(50)
novr_3d_median_diff_df = novr_3d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)
ConductWilcoxon(numeric_cols, novr_3d_median_diff_df)

Delta_AF7	2Sided: 25.0/0.02496337890625 	|Greater: 25.0/0.9893035888671875  	|Less: 25.0/0.012481689453125
Delta_AF8	2Sided: 19.0/0.009185791015625 	|Greater: 19.0/0.996185302734375  	|Less: 19.0/0.0045928955078125
Delta_TP9	2Sided: 15.0/0.004180908203125 	|Greater: 15.0/0.998321533203125  	|Less: 15.0/0.0020904541015625
Delta_TP10	2Sided: 1.0/6.103515625e-05 	|Greater: 1.0/0.9999847412109375  	|Less: 1.0/3.0517578125e-05
Theta_AF7	2Sided: 58.0/0.632171630859375 	|Greater: 58.0/0.7017059326171875  	|Less: 58.0/0.3160858154296875
Theta_AF8	2Sided: 42.0/0.19281005859375 	|Greater: 42.0/0.912322998046875  	|Less: 42.0/0.096405029296875
Theta_TP9	2Sided: 27.0/0.033538818359375 	|Greater: 27.0/0.985504150390625  	|Less: 27.0/0.0167694091796875
Theta_TP10	2Sided: 7.0/0.000579833984375 	|Greater: 7.0/0.999786376953125  	|Less: 7.0/0.0002899169921875
Alpha_AF7	2Sided: 61.0/0.743560791015625 	|Greater: 61.0/0.6471405029296875  	|Less: 61.0/0.3717803955078125
Alpha_AF8	2Sided: 54.0/0.49542236328

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\3261009585.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  novr_3d_median_diff_df = novr_3d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)


__Two-Sided__: I think it's worth declaring that there's good reason to reject the null hypothesis (that there isn't a difference between novr rest and 3D) for the following:

* All Delta bands
* All Theta except AF7
* Only TP channels for Alpha
* Only AF channels for Beta
* All Gamma channels

__Greater__: What's very interesting is that we cannot reject the null hypothesis for a lot of these. Here's ones where we can reject the null hypothesis:

* AF channels for Beta band
* All of Gamma band

__Lesser__: We reject the null hypothesis for these:

* All Delta
* Only TP channels for Theta
* Only TP channels for Alpha

This is a very interesting development here. The concentration task effectively shows that there is a two-sided difference for Delta and Theta, that there is a greater median difference in the AF channels for beta and all of Gamma, and a lesser median difference in the Delta band and TP channels for Theta and Alpha. I think there's a somewhat obvious thing: We see an increase in Beta and Gamma predominantly, and we see a decrease in the Delta, Theta, and Alpha (or at least some of their channels).

## Comparing HMD Rest and 3D

In [54]:
vr_3d_median_df = rest_conc_median_df[rest_conc_median_df['task_type'].isin(["vr","3d"])]
vr_3d_median_sorted_df = vr_3d_median_df.sort_values(['participant','task'],ascending=False).groupby('participant').head(50)
vr_3d_median_diff_df = vr_3d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)
ConductWilcoxon(numeric_cols, vr_3d_median_diff_df)

Delta_AF7	2Sided: 20.0/0.010986328125 	|Greater: 20.0/0.9954071044921875  	|Less: 20.0/0.0054931640625
Delta_AF8	2Sided: 12.0/0.00213623046875 	|Greater: 12.0/0.9991607666015625  	|Less: 12.0/0.001068115234375
Delta_TP9	2Sided: 11.0/0.001678466796875 	|Greater: 11.0/0.9993438720703125  	|Less: 11.0/0.0008392333984375
Delta_TP10	2Sided: 0.0/3.0517578125e-05 	|Greater: 0.0/1.0  	|Less: 0.0/1.52587890625e-05
Theta_AF7	2Sided: 41.0/0.17535400390625 	|Greater: 41.0/0.92047119140625  	|Less: 41.0/0.087677001953125
Theta_AF8	2Sided: 38.0/0.129730224609375 	|Greater: 38.0/0.9416656494140625  	|Less: 38.0/0.0648651123046875
Theta_TP9	2Sided: 25.0/0.02496337890625 	|Greater: 25.0/0.9893035888671875  	|Less: 25.0/0.012481689453125
Theta_TP10	2Sided: 12.0/0.00213623046875 	|Greater: 12.0/0.9991607666015625  	|Less: 12.0/0.001068115234375
Alpha_AF7	2Sided: 25.0/0.02496337890625 	|Greater: 25.0/0.9893035888671875  	|Less: 25.0/0.012481689453125
Alpha_AF8	2Sided: 34.0/0.083251953125 	|Greater: 34.0/0

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\3098552534.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vr_3d_median_diff_df = vr_3d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)


These results show to me some very interesting results.

__Two-Sided__: We reject the null hypothesis (that there isn't a difference between novr rest and 3D) for the following:

* All Delta bands
* Only TP channels for Theta
* All Alpha channels except for AF8
* Only AF7 for Gamma

__Greater__: We reject the null hypothesis for:

* Only AF channels for Gamma

__Lesser__: We reject the null hypothesis for these:

* All AF and TP9 channels for Delta
* Only TP channels for Theta
* Only AF and TP9 channels for Alpha

This is a very interesting development here. We don't see any changes in Beta, like we did with the NoVR vs 3D case. HOWEVER, there is a trend in how the delta, theta, and alpha are decreasing. The Gamma channel still increasees though. So in a way, oddly enough, we see a very reduced version of behavior seen in NoVR vs 3D. It can be presumed therefore that the introducing of wearing the HMD increased beta across the board, but when it came to actual concentration we see only Gamma increase significantly. Across both NoVR and VR, the Delta, Theta, and Alpha see a significant decrease, though there is some differences in the channels themselves.

## Comparing Rest and 2D

In [60]:
novr_2d_median_df = rest_conc_median_df[rest_conc_median_df['task_type'].isin(["novr","2d"])]
novr_2d_median_sorted_df = novr_2d_median_df.sort_values(['participant','task'],ascending=False).groupby('participant').head(50)
novr_2d_median_diff_df = novr_2d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)
print("NoVR vs 2D")
ConductWilcoxon(numeric_cols, novr_2d_median_diff_df)
print("----------------\n")

vr_2d_median_df = rest_conc_median_df[rest_conc_median_df['task_type'].isin(["vr","2d"])]
vr_2d_median_sorted_df = vr_2d_median_df.sort_values(['participant','task'],ascending=False).groupby('participant').head(50)
vr_2d_median_diff_df = vr_2d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)
print("VR vs 2D")
ConductWilcoxon(numeric_cols, vr_2d_median_diff_df)

NoVR vs 2D
Delta_AF7	2Sided: 4.0/0.000213623046875 	|Greater: 4.0/0.9999237060546875  	|Less: 4.0/0.0001068115234375
Delta_AF8	2Sided: 13.0/0.002685546875 	|Greater: 13.0/0.998931884765625  	|Less: 13.0/0.0013427734375
Delta_TP9	2Sided: 26.0/0.02899169921875 	|Greater: 26.0/0.987518310546875  	|Less: 26.0/0.014495849609375
Delta_TP10	2Sided: 9.0/0.001007080078125 	|Greater: 9.0/0.9996185302734375  	|Less: 9.0/0.0005035400390625
Theta_AF7	2Sided: 47.0/0.2978515625 	|Greater: 47.0/0.8627777099609375  	|Less: 47.0/0.14892578125
Theta_AF8	2Sided: 48.0/0.322509765625 	|Greater: 48.0/0.85107421875  	|Less: 48.0/0.1612548828125
Theta_TP9	2Sided: 31.0/0.05767822265625 	|Greater: 31.0/0.97467041015625  	|Less: 31.0/0.028839111328125
Theta_TP10	2Sided: 24.0/0.021392822265625 	|Greater: 24.0/0.990875244140625  	|Less: 24.0/0.0106964111328125
Alpha_AF7	2Sided: 58.0/0.632171630859375 	|Greater: 58.0/0.7017059326171875  	|Less: 58.0/0.3160858154296875
Alpha_AF8	2Sided: 63.0/0.820892333984375 	|Great

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\244462168.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  novr_2d_median_diff_df = novr_2d_median_sorted_df.groupby(['participant'], as_index=False).apply(subtract_rows)
C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_10372\244462168.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vr_2d_median_diff_df = vr_2d_median_sorted_df.groupby(['participant

### No-VR vs 2D:

__Two-Sided__: We reject the null hypothesis (that there isn't a difference between novr rest and 3D) for the following:

* __All Delta bands__
* Only TP10 for Theta
* _Only TP channels for Alpha_
* Only AF channels for Beta
* _All of Gamma_

__Greater__: We reject the null hypothesis for:

* _Only AF channels for Beta_
* _All of Gamma_

__Lesser__: We reject the null hypothesis for these:

* __All of Delta__
* _Only TP channels for Theta_
* _Only TP channels for Alpha_

### VR vs 2D:

__Two-Sided__: We reject the null hypothesis (that there isn't a difference between novr rest and 3D) for the following:

* __All Delta bands__
* Only AF8 and TP9 for Theta
* _All except AF8 for Alpha_
* _Only AF channels for Gamma_

__Greater__: What's very interesting is that we cannot reject the null hypothesis for a lot of these. Here's ones where we can reject the null hypothesis:

* _Only AF8 for Beta_
* _Only AF channels for Gamma_

__Lesser__: We reject the null hypothesis for these:

* __All Delta__
* _All except TP10 for Theta_
* _All except AF8 for Alpha_

Okay, so here's the apparent comparison I'm identifying here. There are observable trends in the transition from rest to concentration. A lot of greater increases are in the Beta and Gamma bands, and a lot of decreases in the Delta, Theta, and Alpha bands. This makes sense upon review - we know that the Beta and Gamma are responsible for cognitive concentration, executive functions, and attention. This appears to be a good correlation, though there isn't a complete overlap in channels. The best we can really do is point this out as an identifiable correlation that demands exploration to an extent.

# Combining all Wilcoxon Test Results

I think there's a general trend we can observe across all the data. For the sake of consistency, we emphasize:

* We assume a 5% confidence level for all statistica p-values
* There are 3 null hypotheses:
    1. __2-Sided__: There is no difference in log power between <CONDITION 1> and <CONDITION 2>
    2. __Greater__: There is no median difference greater than 0 in the log power between <CONDITION 1> and <CONDITION 2>
    3. __Less__: There is no median difference lesser than 0 in the log power between <CONDITION 1> and <CONDITION 2>

## 1. HMD Presence during Rest Task

__2-Sided__: We reject the null hypothesis that the wearing of the HMD does not affect the log power of the EEG signals for the selected data:

* Delta AF7
* Alpha AF7
* Beta AF8
* Gamma TP9, TP10

__Greater__: We reject the null hypothesis that the mediaan difference in the log power between wearing and not wearing the HMD for the selected data:

* Delta AF7
* Alpha AF7
* Beta AF7, AF8
* Gamma TP9, TP10

__Less__: We cannot reject the null hypothesis that the median difference in the log power is smaller than 0. No channels produce a p-value less than 5%.

__Insights__: Given that we are primarily interested in the Alpha, Beta, and Gamma bands, the fact that we DO get a response from those bands is somewhat promising. What is a little weird though is that the alpha and delta only have AF7, beta only has AF8, and gamma is... gamma for TP9 and TP10. We don't really want to explore TP9 or TP10, so this is a little curious. I think though it can be said at least that the EEG data for TP9 and TP10 is at least stable. Or at least I would like to think so. They're certainly more stable than the real-world condition.


## 2. 3D vs. 2D Concentration

__2-Sided__: We reject the null hypothesis that the wearing of the HMD does not affect the log power of the EEG signals for the selected data:

* ... basically everything except Theta TP10 and Alpha TP10

__Greater__: We cannot reject the null hypothesis that the median difference in log power is greater than 0. No channels produce a p-value less than 5%

__Less__: We reject the null hypothesis that the median difference in log power is smaller than 0 for the following channels:

* Theta TP10
* Alpha AF8, TP10

__Insights__: Note that Theta is the only frequency band that has some response here, so far. Furthermore, it's specifically in the TP10 channel. The same response is seen in Alpha TP10. What's very interesting is Alpha AF8, which also does not show up when we looked at the rest state conditions.

## 3. No-HMD vs. 3D

__Two-Sided__: We reject the null hypothesis that the 3D concentration task was different from the Non-HMD rest state for the following channels:

* All Delta bands
* Theta AF8, TP9, TP10
* Alpha TP9, TP10
* Beta AF7, AF8
* All Gamma channels

__Greater__: We reject the null hypothesis for the following channels:

* Beta AF7, AF8
* All Gamma channels

__Less__: We reject the null hypothesis for the following channels:

* All Delta
* Theta TP9, TP10 
* Alpha TP9, TP10

__Insights__: The concentration task effectively shows that there is a two-sided difference for Delta and Theta, that there is a greater median difference in the AF channels for beta and all of Gamma, and a lesser median difference in the Delta band and TP channels for Theta and Alpha. I think there's a somewhat obvious thing: We see an increase in Beta and Gamma predominantly, and we see a decrease in the Delta, Theta, and Alpha (or at least some of their channels).

## 4. HMD vs 3D

__Two-Sided__: We reject the null hypothesis for the following channels:

* All Delta bands
* Theta TP9, TP10
* Alpha AF7, TP9, TP10
* Gamma AF7

__Greater__: We reject the null hypothesis for the following channels:

* Gamma AF7, AF8

__Less__: We reject the null hypothesis for the following channels:

* Delta AF7, AF8, TP9
* Theta TP9, TP10
* Alpha AF7, AF8, TP9

__Insights__: We don't see any changes in Beta, like we did with the NoVR vs 3D case. HOWEVER, there is a trend in how the delta, theta, and alpha are decreasing. The Gamma channel still increasees though. So in a way, oddly enough, we see a very reduced version of behavior seen in NoVR vs 3D. It can be presumed therefore that the introducing of wearing the HMD increased beta across the board, but when it came to actual concentration we see only Gamma increase significantly. Across both NoVR and VR, the Delta, Theta, and Alpha see a significant decrease, though there is some differences in the channels themselves.

## 5. No-HMD vs 2D

__Two-Sided__: We reject the null hypothesis for the following channels:

* All of Delta
* Theta TP10
* Alpha TP9, TP10
* Beta AF7, AF8
* All of Gamma

__Greater__: We reject the null hypothesis for the following channels:

* Beta AF7, AF8
* All of Gamma

__Less__: We reject the null hypothesis for the following channels:

* All of Delta
* Theta TP9, TP10
* Alpha TP9, TP10

## 6. HMD vs 2D:

__Two-Sided__: We reject the null hypothesis for the following channels:

* All Delta bands
* Theta AF8, TP9
* Alpha AF7, TP9, TP10
* Gamma AF7, AF8

__Greater__: We reject the null hypothesis for the following channels:

* Beta AF8
* Gamma AF7, AF8

__Less__: We reject the null hypothesis for the following channels:

* All Delta
* Theta AF7, AF8, TP9
* Alpha AF7, TP9, TP10

There are observable trends in the transition from rest to concentration. A lot of greater increases are in the Beta and Gamma bands, and a lot of decreases in the Delta, Theta, and Alpha bands. This makes sense upon review - we know that the Beta and Gamma are responsible for cognitive concentration, executive functions, and attention. This appears to be a good correlation, though there isn't a complete overlap in channels. The best we can really do is point this out as an identifiable correlation that demands exploration to an extent.

## Final Discussion

It would appear that wearing the HMD during the resting task produces a greater log power in individuals than when not wearing the HMD. The response is statistically significant in the AF channels for the most part. However, the change between 3D and 2D did result in many statistically significant differences - only in the TP10 channel in the Theta and Alpha, and AF8 in the Alpha band. In other words, the wearing of the HMD produced a strong effect during resting as a baseline affect.

When looking across the differences between resting and concentrating, what's incredibly important is the Non-HMD to Concentration, both 3D and 2D red ball. The statistically significant increase in log power is visible in both the AF channels in Beta and all across Gamma. Conversely, there are statistically significant differences in the Delta, Theta, and Alpha channels. This makes sense, given our general comprehension of the behavior associated with these frequency bands. When we go from HMD to concentration, however, the only statistically significant increase is in the AF channels in the Gamma band. Statistically significant decreases are identified in the Delta, Theta, and Alpha, similar to the non-HMD condition. 

I think what this emphasizes is that when wearing the HMD, there is an increase in Beta activity. This increase in beta activity is not seen if we go form HMD to concentration task, but is seen from non-HMD to concentration. Rather, we see an increase in the Gamma across either HMD or non-HMD condition. So __the introduction of HMD automatically creates a Beta activity increase__. This must be accounted for in later analysis.

Also important to note is that there is no relevant statistical difference in the median differences between 3D and 2D concentration. In other words, I don't think it's really important to differentiate between whether to use 3D or 2D as a median. What's very important though is that __if we want to use a resting state as a median to normalize under, it's optimal to do so with the HMD resting condition. This is because there's a recognizable difference in the non-HMD and HMD conditions and since the walking task is conducted in VR, it's perhaps more optimal to normalize under the HMD condition.__

I think the final question is whether we want to use a resting or a concentration median to derive our analysis off of. For the most part, I think it's optimal to use a resting state instead of a concentration state. The concentration response is what we're looking for in the data. If we normalize the trial data to the concentration response, then that means it's likely possible that we won't actually identify any basic concentration response. If anything, we would be focusing our analysis on identifying resting states. So __it's perhaps more optimal to use the HMD Resting median as a normalization factor__.